# Example of rAIse utilisation (MNIST data)

In [ ]:
%pylab inline

import json
import os
import sys
from configparser import ConfigParser

import dataiku
from keras import backend as K
from keras.utils import to_categorical


CONNECTOR_DIR = os.path.join(sys.path[0], "connector")
sys.path.append(CONNECTOR_DIR)

## Preprocess data

In [ ]:
mnist_data = dataiku.Dataset("mnist_normalised").get_dataframe()
labels_OHE = dataiku.Dataset("labels_one_hot_encoding").get_dataframe()

num_classes = 10

# Input image dimensions
img_rows, img_cols = 28, 28

In [ ]:
x_train = mnist_data.values[:10000]
y_train = labels_OHE.values[:10000]
x_test = mnist_data.values[10000:11500]
y_test = labels_OHE.values[10000:11500]

if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Create initial model (optional)

In [ ]:
import random

from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D
from keras.models import Model


batch_size = 100
epochs = 5

input_layer = Input(shape=input_shape)
x = Conv2D(filters=random.choice(range(8, 129, 8)),
           kernel_size=random.choice(range(3, 11)),
           activation="relu",
           kernel_initializer="he_normal")(input_layer)
x = MaxPooling2D(pool_size=random.choice(range(2, 5)))(x)
x = Flatten()(x)
output_layer = Dense(num_classes,
                     kernel_initializer="he_normal",
                     activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))

In [ ]:
# Save model
path_dir = os.path.join(CONNECTOR_DIR, "InitialModel")
if not os.path.exists(path_dir):
    os.makedirs(path_dir)

model.save_weights(os.path.join(path_dir, "model_weights.h5"), overwrite=True)
with open(os.path.join(path_dir, "model.config"), "w") as f:
    json.dump(model.to_json(), f)

## rAIse integration

### Trainer object

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from examples.itrainer import ITrainer


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


class Trainer(ITrainer):
    @staticmethod
    def save_model(model, path_dir="../InitialModel"):
        if not os.path.exists(path_dir):
            os.makedirs(path_dir)
        model.save_weights(os.path.join(
            path_dir, "model_weights.h5"), overwrite=True)
        with open(os.path.join(path_dir, "model.config"), "w") as f:
            json.dump(model.to_json(), f)

    def create_model(self):
        pass
    
    def train_func(self, model):
        batch_size = 100
        epochs = 5

        model.compile(loss="categorical_crossentropy",
                      optimizer="Adam",
                      metrics=["accuracy"])

        early_stop = EarlyStopping(monitor="val_loss",
                                   min_delta=0,
                                   patience=1,
                                   verbose=0,
                                   mode="auto")
        reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                                      factor=0.2,
                                      patience=1,
                                      min_lr=0.001)
        results = model.fit(x_train,
                            y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[early_stop, reduce_lr])

        train_accuracy = results.history["accuracy"][-1]
        test_accuracy = results.history["val_accuracy"][-1]

        return train_accuracy, test_accuracy

    def evaluate_func(self, model):
        batch_size = 200

        model.compile(loss="categorical_crossentropy",
                      optimizer="Adam",
                      metrics=["accuracy"])
        results = model.evaluate(x_test, y_test, batch_size=batch_size)

        return results[-1]

### Model optimization

In [ ]:
import run_population as rp

config_file = os.path.join(CONNECTOR_DIR, "project.cfg")
project_config = ConfigParser()
project_config.read(config_file)
trainer_class = Trainer

cwd = os.getcwd()
os.chdir(CONNECTOR_DIR)

try:
    rp.run_population(project_config=project_config,
                      trainer_class=trainer_class)
except SystemExit:
    pass
finally:
    os.chdir(cwd)

### Get champion and use it

In [ ]:
from keras.models import model_from_json

from clients.api import RequestToApi


config_file = os.path.join(CONNECTOR_DIR, "project.cfg")
project_config = ConfigParser()
project_config.read(config_file)
url = project_config.get("DEFAULT", "url")
project_id = project_config.get("DEFAULT", "project_id")
token = project_config.get("DEFAULT", "token")
agents_dir = project_config.get("DEFAULT", "agents_path")

# Get champion ID
api = RequestToApi(url, token)
project = api.get_project(project_id=project_id)
mutant_id = project["scoreChampionId"] if project["scoreChampionId"] else project["initialMutantId"]
champion_id = api.get_mutant(mutant_id=mutant_id)["agentId"]

# Get champion config & weights files
champion_path = os.path.join(CONNECTOR_DIR, agents_dir, project_id, champion_id)
champion_weights = f"{champion_path}_weights.h5"
champion_cfg = f"{champion_path}.config"

with open(champion_cfg, "r") as f:
    champion_config = json.load(f)

# Load network
model = model_from_json(champion_config)
model.load_weights(champion_weights)

In [ ]:
# New data
labels = dataiku.Dataset("labels").get_dataframe()

x_sample = mnist_data.values[-1000:]
y_sample = labels.values[-1000:]

if K.image_data_format() == "channels_first":
    x_sample = x_sample.reshape(x_sample.shape[0], 1, img_rows, img_cols)
else:
    x_sample = x_sample.reshape(x_sample.shape[0], img_rows, img_cols, 1)

prediction = model.predict(x_sample)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix


y_pred = np.argmax(prediction, axis=1)
y_pred = y_pred.reshape(y_pred.shape[0], 1)

conf = confusion_matrix(y_sample, y_pred)
sns.heatmap(conf, annot=True, fmt="d")
plt.show()